In [0]:
# Installing TuriCreate. This is our main package. TuriCreate is open sourced. 
# You can get more info about TuriCreate at https://github.com/apple/turicreate

!pip install turicreate

In [2]:
# Finding the directory we are in
!ls

datalab


Next code block will mount your google drive. This will help you to access files stored in your google drive. 
Druing installation you see a prompt with a link -Please, open the following URL in a web browser: https://accounts.google.com/o ....
Click the link. Copy and paste the code from the link in the form field and press enter. 
Sometime You may have to run this code block twice. I think this is a bug in colab. 

In [0]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

In [0]:
# Mount your google drive to access files from it 
!mkdir -p drive
!google-drive-ocamlfuse drive

In [0]:
# The image folder that I am using for this tutorial is located in my google drive under 'app' folder
# So we need to change directory to the 'app' folder

import os
os.chdir("drive/app")

In [7]:
# make sure that we are in the right folder 
!ls

cats-dogs.sframe  ImageFolder  MelanomaImages  wisconsin_breast_cancer.csv


In [0]:
# importing turicreate as tc

import turicreate as tc

In [0]:
# Loading the images from MelanomaImages in my google drive. This 'MelanomaImages' has 2 folders
# 'melanoma' and 'benign'


data = tc.image_analysis.load_images('MelanomaImages', with_path=True)

In [0]:
# Labelling each images in the SFrame using the folder name

data['label'] = data['path'].apply(lambda path: 'melanoma' if '/melanoma' in path else 'benign')

In [12]:
#Displaying the first 5 rows in data to make sure that we got everything right 

data.tail()

path,image,label
/content/drive/app/MelanomaImages/melanoma/ISI ...,Height: 4439 Width: 6708,melanoma
/content/drive/app/MelanomaImages/melanoma/ISI ...,Height: 4441 Width: 6661,melanoma
/content/drive/app/MelanomaImages/melanoma/ISI ...,Height: 4401 Width: 6681,melanoma
/content/drive/app/MelanomaImages/melanoma/ISI ...,Height: 4459 Width: 6627,melanoma
/content/drive/app/MelanomaImages/melanoma/ISI ...,Height: 4439 Width: 6668,melanoma
/content/drive/app/MelanomaImages/melanoma/ISI ...,Height: 4419 Width: 6648,melanoma
/content/drive/app/MelanomaImages/melanoma/ISI ...,Height: 4422 Width: 6661,melanoma
/content/drive/app/MelanomaImages/melanoma/ISI ...,Height: 4420 Width: 6641,melanoma
/content/drive/app/MelanomaImages/melanoma/ISI ...,Height: 4441 Width: 6640,melanoma
/content/drive/app/MelanomaImages/melanoma/ISI ...,Height: 4459 Width: 6708,melanoma


In [0]:
# Saving the data frame to our google drive 
data.save('melanoma.sframe')

In [0]:
# Splitting the image data set into training (80%) and testing (20%) data
train_data, test_data = data.random_split(0.8)

In [0]:
# Switching to the root directory in our google virtual machine
os.chdir("/usr")

In [0]:
# installing other packahes to use GPU
!apt install libnvrtc8.0

In [17]:
# uninstalling previous versions of mxnet
!pip uninstall -y mxnet

Uninstalling mxnet-0.12.1:
  Successfully uninstalled mxnet-0.12.1


In [18]:
# Installing the latest version of mxnet to work with turi create
!pip install mxnet==1.1.0

    100% |################################| 23.8MB 60kB/s 
    100% |################################| 17.0MB 84kB/s 
  Found existing installation: graphviz 0.8.2
    Uninstalling graphviz-0.8.2:
      Successfully uninstalled graphviz-0.8.2
  Found existing installation: numpy 1.14.2
    Uninstalling numpy-1.14.2:
      Successfully uninstalled numpy-1.14.2


In [0]:
# making sure that mxnet is working
# If you get an error here, you need to fix it before going forward
import mxnet as m2

In [20]:
# Automatically pick the right model based on your data.
# Note: Because the dataset is large, model creation may take hours.

model = tc.image_classifier.create(train_data, target='label')

Download completed: /var/tmp/model_cache/resnet-50-symbol.json
Download completed: /var/tmp/model_cache/resnet-50-0000.params
Resizing images...
Performing feature extraction on resized images...
Completed 512/734
Completed 734/734
PROGRESS: Creating a validation set from 5 percent of training data. This may take a while.
          You can set ``validation_set=None`` to disable validation tracking.



WARNING: The number of feature dimensions in this problem is very large in comparison with the number of examples. Unless an appropriate regularization value is set, this model may not provide accurate predictions for a validation/test set.

WARNING: Detected extremely low variance for feature(s) '__image_features__' because all entries are nearly the same.
Proceeding with model training using all features. If the model does not provide results of adequate quality, exclude the above mentioned feature(s) from the input dataset.

Logistic regression:

--------------------------------------------------------

...

| 6         | 15       | 1.000000  | 2.066136     | 0.723891          | 0.685714            |

| 10        | 20       | 1.000000  | 2.569017     | 0.831187          | 0.828571            |

+-----------+----------+-----------+--------------+-------------------+---------------------+

TERMINATED: Iteration limit reached.

This model may not be optimal. To improve it, consider increasing `max_iterations`.

Took only about 7 minutes to do this with GPU!

In [0]:
# storing the prediction 

predictions = model.predict(test_data)

In [23]:
# Evaluating the model 

metrics = model.evaluate(test_data)
print(metrics['accuracy'])

Resizing images...
Performing feature extraction on resized images...
Completed 183/183
0.825136612021858


In [24]:
# Printing confusion matrix

print ("Confusion Matrix : \n%s" % metrics['confusion_matrix'])

Confusion Matrix : 
+--------------+-----------------+-------+
| target_label | predicted_label | count |
+--------------+-----------------+-------+
|    benign    |      benign     |  102  |
|    benign    |     melanoma    |   7   |
|   melanoma   |     melanoma    |   49  |
|   melanoma   |      benign     |   25  |
+--------------+-----------------+-------+
[4 rows x 3 columns]




```

> Statistic           Value            Confidence interval 


> Sensitivity         66.22%	        54.28% to 76.81%



> Specificity         93.58 %	       87.22% to 97.38%


> PPV                 87.50%    	    77.05% to 93.59%


> NPV                 80.31 %           74.71% to 84.93%


> Accuracy            82.51% 	       76.22% to 87.72%


```

In [0]:
# Save the model for later use in Turi Create
model.save('melanoma.model')

In [0]:
# Export for use in Core ML
model.export_coreml('MyMelanomaClassifier.mlmodel')

In [27]:
!ls

MyMelanomaClassifier.mlmodel  games    lib	     local	     sbin   src
bin			      include  lib64-nvidia  melanoma.model  share


In [0]:
# Downloading the model for use in Xcode

from google.colab import files

files.download('MyMelanomaClassifier.mlmodel')